In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Yola_fit').getOrCreate()



In [2]:
df = spark.read.csv('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Pricing_fallback_rule.csv', header='True')



In [3]:
df.createOrReplaceTempView('Fallback_rule')

In [4]:
df.show()

+------------------+----------+--------------------+--------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+---+-----------------+---------------+
|         item_code|      date|item_gtin_comparable|  item_main_category| item_sub_category_1|item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin|nov|shop_gross_margin|shop_pc2_margin|
+------------------+----------+--------------------+--------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+---+-----------------+---------------+
|000000001000052666|2020-02-16|          comparab

In [5]:
import pandas as pd
import numpy as np
import patsy as pat



In [6]:
pdf_sap = pd.read_excel('/Users/gabriele.sabato/PycharmProjects/raw_data/Excel_files/Lieferanten_Master_Data_SAP_V2.xlsx',header=3, usecols=['SAP_ID','Supplier_Name','Type'])#,'Type'])
pdf_sap.dropna(inplace=True)
print(pdf_sap.head(100))


#df_sap = spark.createDataFrame(pdf_sap)



       SAP_ID                         Supplier_Name         Type
2    100000.0                    Art of Living GmbH  Marketplace
3    100001.0            Actona Company AS (Inland)     Domestic
4    100002.0   Actona Company A/S (Hauptwerk SHANG   FOB Import
5    100003.0                      Actual Diffusion  Marketplace
6    100004.0                   Arte M GmbH & Co.KG     Domestic
..        ...                                   ...          ...
103  100109.0   Duo Collection Import Vertriebsgese     Domestic
104  100110.0                          David & Luna     Domestic
105  100111.0  design3000 Vertriebsgesellschaft mbH  Marketplace
106  100112.0             Dreisitz-design Sp.z.o.o.     Domestic
107  100113.0                          Dan-Form A/S     Domestic

[100 rows x 3 columns]


In [7]:
print(pdf_sap.dtypes)
pdf_sap_filt = pdf_sap.astype({'SAP_ID': 'int32', 'Supplier_Name': 'string','Type':'string'})
print(pdf_sap_filt.dtypes)



SAP_ID           float64
Supplier_Name     object
Type              object
dtype: object
SAP_ID            int32
Supplier_Name    string
Type             string
dtype: object


In [8]:
df_sap = spark.createDataFrame(pdf_sap_filt)
df_sap.createOrReplaceTempView('SAP_id')

In [9]:
df_sap_INJOIN = spark.sql("""SELECT fbr.*, sap.Type
                                from Fallback_rule as fbr
                              INNER JOIN SAP_id as sap on sap.SAP_ID = CAST(fbr.item_supplier_regular_id as INT) """)

In [10]:
df_sap_INJOIN.createOrReplaceTempView('IN_JOIN_Table')

In [11]:
df_sap_INJOIN.show()

+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+
|000000001000097652|20

In [25]:
df_fin_table = spark.sql(""" SELECT ijt.*, shop_pc2_margin/nov  as shop_pc2_margin_nov_ratio,
        CASE
           when ijt.item_inco_terms = 'FOB'
               then 'Import'
           when LOWER(ijt.Type) LIKE 'eastern europe'
               then 'EE'
           when LOWER(ijt.Type) LIKE 'import'
               then 'Import'
           else 'Domestic' END                                                              as sourcing_location
            from IN_JOIN_Table as ijt
""")
#shop_pc2_margin/NOV


In [26]:
df_fin_table.show()
df_fin_table.createOrReplaceTempView('Fin_table')

+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+-------------------------+-----------------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1| item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|shop_pc2_margin_nov_ratio|sourcing_location|
+------------------+----------+--------------------+------------------+--------------------+--------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-

In [75]:
df_test = spark.sql("""SELECT DISTINCT exclusivity_clean from Fin_table """)

In [76]:
df_test.show()

+-----------------+
|exclusivity_clean|
+-----------------+
|        exclusive|
|            other|
|       thirdParty|
|     ownDeveloped|
|       whiteLabel|
+-----------------+



In [29]:
df_item_dates = spark.sql("""SELECT * from Fin_table where item_code = '000000001000097478' ORDER BY date ASC""")
df_item_dates.show(500)

+------------------+----------+--------------------+------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+-----------------+----+-----------------+---------------+--------+-------------------------+-----------------+
|         item_code|      date|item_gtin_comparable|item_main_category| item_sub_category_1|item_sub_category_2|item_supplier_regular_id|item_inco_terms|exclusivity_clean|item_supplier_regular|inventory_flag|wh_flag|item_final_price_rule|sellability_laso_de|pocket_pc2_margin| nov|shop_gross_margin|shop_pc2_margin|    Type|shop_pc2_margin_nov_ratio|sourcing_location|
+------------------+----------+--------------------+------------------+--------------------+-------------------+------------------------+---------------+-----------------+---------------------+--------------+-------+---------------------+-------------------+----

In [30]:
pdf_fin = df_fin_table.toPandas()

In [31]:
#MODEL IMPLEMENTATION
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

list_of_num_columns = ['wh_flag','shop_pc2_margin','pocket_pc2_margin','nov','shop_gross_margin']
for col in list_of_num_columns:
    pdf_fin[col]=pd.to_numeric(pdf_fin[col])
#pdf_fin = pdf_fin.astype({'wh_flag':'int32', 'shop_pc2_margin':'int32','item_main_category':'string','exclusivity_clean':'string','sourcing_location':'string' })


In [32]:
print(pdf_fin.dtypes)


item_code                     object
date                          object
item_gtin_comparable          object
item_main_category            object
item_sub_category_1           object
item_sub_category_2           object
item_supplier_regular_id      object
item_inco_terms               object
exclusivity_clean             object
item_supplier_regular         object
inventory_flag                object
wh_flag                        int64
item_final_price_rule         object
sellability_laso_de           object
pocket_pc2_margin              int64
nov                            int64
shop_gross_margin              int64
shop_pc2_margin                int64
Type                          object
shop_pc2_margin_nov_ratio    float64
sourcing_location             object
dtype: object


In [40]:
WH_with_fe_false = smf.ols("shop_pc2_margin_nov_ratio ~  C(wh_flag) + C(sourcing_location) + C(exclusivity_clean) +C(item_main_category) -1",  pdf_fin,hasconst = False).fit()
print(WH_with_fe_false.summary())
#

                                    OLS Regression Results                                    
Dep. Variable:     shop_pc2_margin_nov_ratio   R-squared (uncentered):                   0.416
Model:                                   OLS   Adj. R-squared (uncentered):              0.416
Method:                        Least Squares   F-statistic:                          1.951e+04
Date:                       Fri, 29 Jan 2021   Prob (F-statistic):                        0.00
Time:                               18:04:14   Log-Likelihood:                     -3.8108e+05
No. Observations:                     712887   AIC:                                  7.622e+05
Df Residuals:                         712861   BIC:                                  7.625e+05
Df Model:                                 26                                                  
Covariance Type:                   nonrobust                                                  
                                                  

In [46]:
parameter_map = zip(list(WH_with_fe_false.params),list(WH_with_fe_false.bse))
print(list(parameter_map))


[(0.2024182964767709, 0.008319919412287833), (0.21145419706931012, 0.008250749108829374), (0.028023331425552876, 0.0021230777106215682), (0.09397785000822635, 0.0015577145536698298), (-0.015631502820477622, 0.0026524182925162483), (0.0008343690598767439, 0.002558931513808), (-0.02797762378578887, 0.002087923669896421), (-0.021606730541537902, 0.0015270514351546556), (0.15893642235004726, 0.008503113840124605), (0.06996807900454585, 0.008351927549905347), (0.038626702232119135, 0.008623481738130078), (0.14437532528300734, 0.008437712930956373), (0.15447062623775237, 0.008264639848775589), (0.13461234721568013, 0.008518842571985883), (0.1478561157298066, 0.008367495776137155), (0.024056654853440357, 0.008910536290498131), (-0.11903492867999198, 0.04659941760110478), (0.0620457815512156, 0.008539323389452068), (0.02250457355436007, 0.009947575058954066), (0.16042572221587145, 0.015297814696699693), (0.099989173712822, 0.008227026387198253), (0.12980806042889712, 0.008572811119966259), (0.

In [110]:
params = pd.Series(WH_with_fe_false.params, name = 'beta')
errors  = pd.Series(WH_with_fe_false.bse, name = 'beta_err')

result_df = pd.concat([params,errors],axis=1)
print(result_df)
#for name, value in ndarray:
#    print(name,value)


                                                     beta  beta_err
C(wh_flag)[0]                                    0.202418  0.008320
C(wh_flag)[1]                                    0.211454  0.008251
C(sourcing_location)[T.EE]                       0.028023  0.002123
C(sourcing_location)[T.Import]                   0.093978  0.001558
C(exclusivity_clean)[T.other]                   -0.015632  0.002652
C(exclusivity_clean)[T.ownDeveloped]             0.000834  0.002559
C(exclusivity_clean)[T.thirdParty]              -0.027978  0.002088
C(exclusivity_clean)[T.whiteLabel]              -0.021607  0.001527
C(item_main_category)[T.BAD]                     0.158936  0.008503
C(item_main_category)[T.BETTEN]                  0.069968  0.008352
C(item_main_category)[T.BOUTIQUE]                0.038627  0.008623
C(item_main_category)[T.BÜRO]                    0.144375  0.008438
C(item_main_category)[T.ESSZIMMERSTÜHLE&BÄNKE]   0.154471  0.008265
C(item_main_category)[T.ESSZIMMERTISCHE]        

In [190]:
fin_coeff = pd.DataFrame()

def calc_beta(df =result_df, map = { 'wh_flag':'0', 'sourcing_location' : 'T.EE', 'exclusivity_clean' : 'T.other', 'item_main_category' :'T.GARTEN'}):
    prefix = 'C('
    suffix = ']'

    for key in map.keys():
        map[key]= prefix+ key + ')[' + map[key] + suffix
    #print(map)
    df_filt = pd.DataFrame()
    #print(list(map.values()))
    df_filt = df.loc[list(map.values())]
    print(df_filt)


    print('Final coefficient = ', df_filt['beta'].sum().round(3), '±', np.sqrt((df_filt['beta_err']**2).sum()).round(3))
    return (df_filt['beta'].sum().round(3), np.sqrt((df_filt['beta_err']**2).sum()).round(3))


In [191]:
def map_of_parameters(wh_flag ='0', sourcing_location = 'EE', exclusivity_clean = 'other', item_main_category ='GARTEN' ):
    map_of_parameter = {'wh_flag': wh_flag,
                        'sourcing_location' : 'T.'+sourcing_location,
                        'exclusivity_clean' : 'T.'+exclusivity_clean,
                        'item_main_category': 'T.'+item_main_category}
    if sourcing_location == 'Domestic':
        del map_of_parameter['sourcing_location']
    if exclusivity_clean == 'exclusive':
        del map_of_parameter['exclusivity_clean']
    if item_main_category == 'BABY':
        del map_of_parameter['item_main_category']

    print(map_of_parameter)
    return map_of_parameter


In [192]:
results = calc_beta(result_df,map_of_parameters('0','Domestic','exclusive','BABY'))
#print(results)

{'wh_flag': '0'}
                   beta  beta_err
C(wh_flag)[0]  0.202418   0.00832
Final coefficient =  0.202 ± 0.008
(0.202, 0.008)
